In [1]:
import pandas
import numpy
import geopandas


def geojoin(df, shape,
            dataset_target,
            dataset_features_cum,
            dataset_features_stat,
            dataset_features_dyn,
            dataset_features_dyn_ratio
            ):
    """
    Создание датафрейма для географических визуализаций. ВНИМАНИЕ: Ваш датасет должен иметь колонки ISO и COUNTRY. 

    Args:
        df(pd.DataFrame): Датафрейм с Вашими данными.
        shape(gpd.GeoDataFrame): SHP-файл.
        dataset_target(str): название важного признака, ради которого все и затевалось.
        dataset_features_cum(list): Кумулятивные признаки ("Всего заболело" и т.д.).
        dataset_features_stat(list): Статистические признаки ("ВВП на душу населения" и т.д.)
        dataset_features_dyn(list): Признаки события ("Количество участников" и т.д.)
        dataset_features_dyn_ratio(list): Признаки события, для которых нужно посчитать доли ("Доля")

    Returns:
        gpd.GeoDataFrame: Агрегированный по странам датафрейм.
    """

    df_sum = df[[dataset_target, "ISO", "COUNTRY"]].groupby(by=["ISO", "COUNTRY"]).sum()

    for feature in dataset_features_cum:
        df_sum[feature] = df.groupby(["ISO", "COUNTRY"])[feature].last().fillna(method="ffill")

    for feature in dataset_features_stat:
        df_sum[feature] = df.groupby(["ISO", "COUNTRY"])[feature].mean()

    for feature in dataset_features_dyn:
        df_sum[feature] = df.groupby(["ISO", "COUNTRY"])[feature].sum()

    for feature in dataset_features_dyn_ratio:
        df_sum[feature + "_RATIO"] = df.groupby(["ISO", "COUNTRY"])[feature].sum() / df.groupby(["ISO", "COUNTRY"])[
            feature].count()

    df_sum.reset_index(inplace=True)
    df_sum["ISO"].astype("int64")

    df_geo = shape[["ISO_N3", "geometry", "NAME"]]
    df_geo.loc[:, "ISO"] = pandas.to_numeric(df_geo["ISO_N3"])  # Делаем столбец ISO одинаковым для обоих датасетов
    df_geo = df_geo.drop("ISO_N3", axis=1)

    # стыковка ISO
    join_dict = df_sum.set_index("COUNTRY")["ISO"].to_dict()
    iso_unstacked = list(set(df_geo["ISO"]) - set(df_sum["ISO"]))

    for iso in iso_unstacked:
        mask = df_geo["ISO"] == iso
        df_geo.loc[mask, "ISO"] = df_geo.loc[mask, "NAME"].map(join_dict).values()

    df_geo = pandas.merge(df_geo, df_sum, on="ISO")
    return df_geo


In [2]:
import geopandas as gpd
import pandas

In [3]:
df = pandas.read_excel('datasets/disorders.xlsx')
df.set_index(df.columns[0], inplace = True)
df['EVENT_DATE'] = pandas.to_datetime(df['EVENT_DATE'])
shape = gpd.read_file("mapshape/ne_110m_admin_0_countries.shp") #файл с картой
df.columns

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/disorders.xlsx'

In [ ]:
import geo_join #Собственный модуль, подробнее о нем в тетрадке 1.5

target = "UNORGANIZED"
ds_cum = ["total_cases", 'total_cases_per_million', "total_deaths", 'total_deaths_per_million']
ds_stat = ['reproduction_rate',
       'stringency_index', 'population', 'population_density', 'median_age',
       'aged_65_older', 'aged_70_older', 'gdp_per_capita',
       'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers',
       'hospital_beds_per_thousand', 'life_expectancy',
       'human_development_index']
ds_dyn = ['FATALITIES']
ds_dyn_ratio = ['UNORGANIZED']

df_geo = geo_join.geojoin(df, shape, target, ds_cum, ds_stat, ds_dyn, ds_dyn_ratio)

In [2]:
import pandas as pd
import numpy as np

In [11]:
df = pd.DataFrame({'B': [1, 2, 3, 4, 5]})
df

,B
0,1
1,2
2,3
3,4
4,5


In [12]:
df["C"] = df.rolling(3, min_periods= 1, closed="left").sum() - df.rolling(2, min_periods= 1, closed="left").sum()

In [13]:
df

,B,C
0,1,NaN
1,2,0.0
2,3,0.0
3,4,1.0
4,5,2.0


In [ ]:
result = seasonal_decompose(unorg_ratio, 
                            model= 'additive',
                            period=365)
result.plot();